<a href="https://colab.research.google.com/github/aluthfian/earth_engine_trial/blob/main/perubahan_suhu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
#ee.Authenticate()
ee.Initialize()

In [2]:
import numpy as np
import folium
from folium import plugins
import branca
from branca.element import Figure
import datetime

In [3]:
#@title ee Mapper folium function
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name, opacity=1):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            opacity = opacity,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [31]:
batas_Indonesia_FC = ee.FeatureCollection("USDOS/LSIB/2017")\
.filter(ee.Filter.eq('COUNTRY_NA', 'Indonesia')).geometry()

batas_provinsi = ee.FeatureCollection("FAO/GAUL/2015/level1")\
.filter(ee.Filter.eq('ADM1_NAME','Papua')\
        .And(ee.Filter.eq('ADM1_NAME','Papua Barat'))).geometry()

batas_wilayah = batas_Indonesia_FC

#data_cuaca = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
#.filterBounds(batas_wilayah)

data_cuaca = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY")\
.filterBounds(batas_wilayah)

#kanal_suhu = 'Tair_f_inst'
kanal_suhu = 'temperature_2m'

tanggal_Awal = '2010-01-01'
tanggal_Akhir = '2022-01-01'

def konversi_ke_celsius(data_raster):
  data_raster = data_raster.subtract(273.15)
  return data_raster

def sysIdx_to_sysTime_A(data_raster):
  indeks = ee.String(data_raster.get('system:index'))
  year = indeks.slice(1, 5).cat('-')
  month = indeks.slice(5, 7).cat('-')
  day = indeks.slice(7, 9).cat('T')
  hour = indeks.slice(10, 12).cat(':')
  menit = indeks.slice(12).cat(':00')
  dateStr = year.cat(month).cat(day).cat(hour).cat(menit)
  std_date = ee.Date(dateStr)
  return data_raster.set("system:time_start", std_date.millis())

def sysIdx_to_sysTime_B(data_raster):
  indeks = ee.String(data_raster.get('system:index'))
  year = indeks.slice(0, 4).cat('-')
  month = indeks.slice(4).cat('-')
  day = ee.String('01T')
  hour = ee.String('00:')
  menit = ee.String('00:00')
  dateStr = year.cat(month).cat(day).cat(hour).cat(menit)
  std_date = ee.Date(dateStr)
  return data_raster.set("system:time_start", std_date.millis())

koleksi_data_suhu = data_cuaca\
.filter(ee.Filter.date(tanggal_Awal, tanggal_Akhir))\
.select(kanal_suhu)\
.map(konversi_ke_celsius)\
.map(sysIdx_to_sysTime_B)

tren_kenaikan_suhu = koleksi_data_suhu.formaTrend(None, 540)
kanal_trend = 'long-trend'
tren_kenaikan_suhu = ee.Image(tren_kenaikan_suhu.select(kanal_trend)).multiply(12)

In [32]:
naikSuhu_tBesar = tren_kenaikan_suhu.reduceRegion(**{
  'reducer':ee.Reducer.max(),
  'geometry': batas_wilayah,
  'scale': 10000,
  'bestEffort': True
})

naikSuhu_tBesar = naikSuhu_tBesar.get(kanal_trend).getInfo();
print(naikSuhu_tBesar);

naikSuhu_tKecil = tren_kenaikan_suhu.reduceRegion(**{
  'reducer':ee.Reducer.min(),
  'geometry': batas_wilayah,
  'scale': 10000,
  'bestEffort': True
})

naikSuhu_tKecil = naikSuhu_tKecil.get(kanal_trend).getInfo();
print(naikSuhu_tKecil);

0.11726799664337524
-0.05098310166564417


In [33]:
skala_warna_temp = {
  ##min: Math.ceil(AOD_tKecil*1000)/1000,
  ##max: 0.1,
  ##bands:['array'],
  'bands':[kanal_trend],
  'min': naikSuhu_tKecil,
  'max': naikSuhu_tBesar,
  ##palet lihat di coolors.co
  'palette': ["#3517fc","#17bebb","#09fd09","#ffc914","#f33823"]
}

# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'OpenTopoMap': folium.TileLayer(
        tiles = 'https://c.tile.opentopomap.org/{z}/{x}/{y}.png',
        attr = 'OpenTopoMap',
        name = 'OSM General',
        overlay = True,
        control = True,
    ),
    'ESRI Shaded Relief': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Shaded_Relief/MapServer/tile/{z}/{y}/{x}',
        attr = 'OpenStreetMap',
        name = 'OSM General',
        overlay = True,
        control = True,
        opacity = 0.5
    ),
    'ESRI Light Grey': folium.TileLayer(
        tiles = 'http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
        attr = 'OpenStreetMap',
        name = 'OSM General',
        overlay = True,
        control = True
    )
}

In [34]:
## Point
bujur = 107.5944
lintang = -6.9115
point = ee.Geometry.Point([bujur, lintang]);

naik_suhu_point = tren_kenaikan_suhu.reduceRegion(**{
  'reducer': ee.Reducer.first(),
  'geometry': point,
  'scale': 10000
})

naik_suhu_point = ee.Number(naik_suhu_point).getInfo()
naik_suhu_point = naik_suhu_point[kanal_trend]

print(naik_suhu_point)

center = batas_wilayah.centroid().getInfo()['coordinates']
center.reverse()

# Create a folium map object.
my_map = folium.Map(location=center, zoom_start=5, width=1100, height=450)

# Add custom basemaps
# basemaps['OSM General'].add_to(my_map)

# Add the LANDSAT data to the map object.
my_map.add_ee_layer(tren_kenaikan_suhu.clip(batas_Indonesia_FC), skala_warna_temp, 'Kenaikan Suhu Tahunan', opacity=0.5)

#specify the min and max values of your data
num_idx_cmap = len(skala_warna_temp['palette'])
index_colormap = np.linspace(naikSuhu_tKecil, naikSuhu_tBesar, num_idx_cmap).tolist()
colormap = branca.colormap.LinearColormap(skala_warna_temp['palette'],
                                          index=index_colormap,
                                          vmin=naikSuhu_tKecil, vmax=naikSuhu_tBesar)
colormap.caption = 'Kenaikan Suhu Tahunan'

svg_style = '<style>svg {background-color: white;}</style>'
my_map.get_root().header.add_child(folium.Element(svg_style))

colormap.add_to(my_map)

teks_suhu = folium.Html("<b>BANDUNG: </b><br> Perubahan suhu <br> tahunan <b>" + '{:.3f}'.format(naik_suhu_point) + "°C </b>",
                        script=True)
popup = folium.Popup(teks_suhu, max_width=95)

folium.Marker(
    location=[lintang, bujur],
    popup=popup,
    icon=folium.Icon(color="orange", icon="info-sign"),
).add_to(my_map)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Add fullscreen button
plugins.Fullscreen().add_to(my_map)

# Display the map.
# display(my_map)
fig = Figure(width=1100, height=450)
svg_style = '<style>svg {background-color: white;}</style>'
fig.get_root().header.add_child(folium.Element(svg_style))
fig.add_child(my_map)

0.04894196653124524
